# DS24 Kunskapskontroll 2 – RAG-baserad chattbot

Notebook med teoretiska svar och en praktisk implementation av en RAG-pipeline.
Kör cellerna uppifrån och ned.

## 1. Miljö & installation
Kör följande för att installera beroenden lokalt. Om du kör i en miljö där du redan har dessa paket kan du hoppa över installationen.

In [ ]:
# Installation (kör vid behov)
# !pip install -q faiss-cpu sentence-transformers langchain langchain-community langchain-core langchain-text-splitters pypdf tiktoken openai python-dotenv
# Om du vill använda OpenAI: skapa en .env med OPENAI_API_KEY


## 2. Importer och konfiguration

In [ ]:
import os
from pathlib import Path

DATA_DIR = Path('docs')
DATA_DIR.mkdir(exist_ok=True, parents=True)

PERSONAL_HANDBOK = DATA_DIR / 'personalhandbok.txt'
if not PERSONAL_HANDBOK.exists():
    PERSONAL_HANDBOK.write_text('Detta är en plats för din personliga handbok. Lägg in dina riktlinjer här.', encoding='utf-8')

print('Datafolder:', DATA_DIR.resolve())
print('Personlig handbok finns:', PERSONAL_HANDBOK.exists())


## 3. Datainläsning & chunking läser in `docs/personalhandbok.txt` (och ev. andra `.txt`/`.pdf`) och delar upp innehållet i mindre segment för att indexeras.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_corpus():
    texts = []
    for p in DATA_DIR.glob('**/*'):
        if p.suffix.lower() == '.txt':
            texts.append(p.read_text(encoding='utf-8', errors='ignore'))
        # För PDF kan pypdf användas, utelämnat här för enkelhet
    return "\n\n".join(texts)

raw_text = load_corpus()
print('Antal tecken i korpus:', len(raw_text))

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, separators=["\n\n", "\n", ". ", " ", ""],
)
chunks = splitter.split_text(raw_text)
print('Antal chunks:', len(chunks))


## 4. Embeddings & vektorindex (FAISS) använder `sentence-transformers` lokalt för att skapa embeddings och bygger ett FAISS-index för snabb sökning.

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model_name = "all-MiniLM-L6-v2"  # liten, snabb
embedder = SentenceTransformer(model_name)

emb = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
index = faiss.IndexFlatIP(emb.shape[1])
faiss.normalize_L2(emb)
index.add(emb)

def retrieve(query, k=5):
    qv = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(qv)
    D, I = index.search(qv, k)
    return [chunks[i] for i in I[0]]

print('Index klart.')


## 5. En enkel RAG-kedja (retrieval + generering)
Här använder en **enkel** generator som bara sammanställer kontext och ger ett svar.

In [ ]:
def simple_answer(query, k=5):
    ctx = retrieve(query, k=k)
    context_text = "\n\n".join([f"- {c}" for c in ctx])
        # Här producerar vi ett deterministiskt svar baserat på kontexten.
    answer = f"Fråga: {query}\n\nKontext (top-{k}):\n{context_text}\n\n" \             f"Sammanfattning: Baserat på kontexten ovan kan vi se nyckelpunkter relaterade till frågan. " \             f"Anpassa gärna till en riktig LLM för bättre svar."
    return answer

print(simple_answer("Vad står i min personliga handbok?")[:500])


## 6. Utvärdering (enkel)
Som en mycket enkel sanity check kör några testfrågor och verifierar att dokumentrelevanta passager hittas.

In [ ]:
test_questions = [
    "Vad är syftet med handboken?",
    "Vilka riktlinjer nämns?"
]
for q in test_questions:
    ctx = retrieve(q, k=3)
    print("\n=====", q, "=====")
    for i, c in enumerate(ctx, 1):
        print(f"[{i}] {c[:200]}...")


## 7. Diskussion och förbättringar
- Byt ut `simple_answer` mot en riktig LLM (OpenAI, Azure OpenAI, Llama.cpp, vLLM etc.).
- Lägg till källcitatsstöd: returnera vilka chunks som användes.
- Lägg till eval med t.ex. *RAGAS* eller manuella Q/A-par.
- Hårda krav på reprod. och mätbara mål: latency, precision@k, faithfulness.

## 8. Teori – korta svar
Nedan är kortfattade, kursrelevanta svar.

### 8.1 Bias–varians och överanpassning
- **Bias**: fel p.g.a. förenklade antaganden; **Varians**: känslighet för träningsdata.
- Överfitting minskas med regularisering (L2, dropout), mer data, data augmentation, tidig stoppning.

### 8.2 Optimerare & inlärningshastighet
- Vanliga optimerare: SGD, Adam, AdamW.\
- Välj `learning rate` via scheman (cosine, step) och `warmup`; för högt ger divergens, för lågt långsam konvergens.

### 8.3 Aktiverings- och förlustfunktioner
- Aktiveringar: ReLU/GeLU (deep nets), Sigmoid/Tanh (sekvens/klassiska).\
- Förluster: CE för klassificering, MSE för regression, triplet/contrastive för representationer.

### 8.4 CNN vs RNN vs Transformer
- **CNN**: bra för bilder/2D-struktur. **RNN**: sekvenser; svårt med långa beroenden. **Transformer**: self-attention, skalbar och SOTA i språk/vision.

### 8.5 RAG: nyckelkomponenter
- Datainhämtning → chunking → embeddings → vektorindex → retriever → generator → citat/utvärdering.\
- Risker: hallucinationer, dålig chunking/metadata, driftsäkerhet.